<a href="https://colab.research.google.com/github/hyehyunseo/ESAA_OB/blob/main/0915_%EB%AA%A8%EB%8D%B8%ED%9B%88%EB%A0%A8_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **| 모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**

[답안]

1. 확률적 경사 하강법

2. 미니배치 경사 하강법

3. 배치 경사 하강법

    +) 정규방정식이나 SVD 방법은 계산 복잡도가 특성 개수에 따라 증가하기 때문에 사용할 수 없음

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**

[답안]

검증 오차가 일정하게 상승하고 있는 것은 학습률이 너무 높아서 검증 오차가 발산하고 있기 때문이다.

* train 에러도 함께 올라간다면, 학습률을 낮춰야함
* train 에러가 올라가지 않는다면 모델이 훈련 세트에 과대적합되고 있는 것이므로 훈련을 멈춰야한다.

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**

[답안]

릿지 회귀를 사용했을 때, 훈련 에러와 검증 에러가 거의 비슷하고 둘다 높다면 모델이 훈련 데이터에 과소적합되었을 가능성이 높다. 다시말해, 이는 높은 편향을 가진 모델이다.

* 따라서, 편향을 줄이기 위해서 alpha를 감소시켜야한다.

### **4. 다음과 같이 사용해야 하는 이유는?**

- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
- 릿지 회귀 대신 라쏘 회귀
- 라쏘 회귀 대신 엘라스틱넷

[답안]

1. 일반적으로 규제가 아예 없는 모델보다 규제가 있는 모델이 성능이 좋다. 따라서, 평범한 선형 회귀보다 릿지 회귀를 사용하는 것이 좋다.

2. 라쏘 회귀는 가장 중요한 가중치를 제외하고는 모두 0이 되는 희소한 모델을 만든다. 또한, 라쏘 회귀는 자동으로 특성 선택의 효과를 가지므로 단지 몇 개의 특성만 실제로 유용할 것이라고 의심될 때 사용하면 좋다.

3. 특성 수가 훈련 샘플 수보다 많거나 특성 몇개가 강하게 연경되어 있는 경우 라쏘가 문제를 일으키므로 라쏘보다는 엘라스틱넷이 선호된다.

### **5. 조기 종료를 사용한 배치 경사 하강법으로 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 데이터 로드
from sklearn import datasets
iris=datasets.load_iris()

In [3]:
X = iris["data"][:, (2, 3)]  # 꽃잎 길이, 꽃잎 넓이
y = iris["target"]

In [4]:
X_with_bias = np.c_[np.ones([len(X), 1]), X]

In [5]:
np.random.seed(1)

In [6]:
test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X_with_bias)

test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - test_size - validation_size

rnd_indices = np.random.permutation(total_size)

X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]
X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]
X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]

In [7]:
def to_one_hot(y):
    n_classes = y.max() + 1
    m = len(y)
    Y_one_hot = np.zeros((m, n_classes))
    Y_one_hot[np.arange(m), y] = 1
    return Y_one_hot

In [8]:
Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot = to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

In [9]:
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

In [10]:
n_inputs = X_train.shape[1]
n_outputs = len(np.unique(y_train))

In [11]:
eta = 0.1
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1  # 규제 하이퍼파라미터
best_loss = np.infty

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients

    logits = X_valid.dot(Theta)
    Y_proba = softmax(logits)
    xentropy_loss = -np.mean(np.sum(Y_valid_one_hot * np.log(Y_proba + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
    loss = xentropy_loss + alpha * l2_loss
    if iteration % 500 == 0:
        print(iteration, loss)
    if loss < best_loss:
        best_loss = loss
    else:
        print(iteration - 1, best_loss)
        print(iteration, loss, "조기 종료!")
        break

0 1.475338742128129
500 0.5300917737500317
1000 0.503409246122055
1500 0.4966005542576664
2000 0.49453281885124234
2500 0.49396775488002026
2872 0.49389591122655674
2873 0.4938959115504358 조기 종료!
